In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Preprocessing

In [2]:
pred_tab = pd.read_csv("combined_predictions_data.csv")
quantum_tab = pred_tab[pred_tab["Raw"] == "quantum"]
data_tab = pred_tab[pred_tab["Raw"] == "raw"]

In [3]:
original = quantum_tab["Posit0"].unique()

In [4]:
for emo in quantum_tab["Emotion"].unique():
    emo_tab = quantum_tab[quantum_tab["Emotion"] == emo]
    count = 0
    original = emo_tab["Posit0"].unique()
    for o in original:
        o_tab = emo_tab[emo_tab["Posit0"] == o]
        for i in o_tab.index:
            quantum_tab.loc[i,"story"] = count
        count = count + 1

C:\Users\Adam\AppData\Local\Temp\ipykernel_8756\1273654453.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantum_tab.loc[i,"story"] = count


In [5]:
for emo in data_tab["Emotion"].unique():
    emo_tab = data_tab[data_tab["Emotion"] == emo]
    count = 0
    original = emo_tab["Posit0"].unique()
    for o in original:
        o_tab = emo_tab[emo_tab["Posit0"] == o]
        for i in o_tab.index:
            data_tab.loc[i,"story"] = count
        count = count + 1

C:\Users\Adam\AppData\Local\Temp\ipykernel_8756\140313429.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tab.loc[i,"story"] = count


In [6]:
quantum_pred_mat = {}
for emo in quantum_tab["Emotion"]:
    emotion_mat = []
    emo_tab = quantum_tab[quantum_tab["Emotion"] == emo]
    for i in emo_tab.index:
        data = []
        data.append(emo_tab.loc[i,"Posit0"])
        data.append(emo_tab.loc[i,"Posit1"])
        data.append(emo_tab.loc[i,"Posit2"])
        data.append(emo_tab.loc[i,"Posit3"])
        data.append(emo_tab.loc[i,"story"])
        emotion_mat.append(data)
    quantum_pred_mat[emo] = np.array(emotion_mat)

In [7]:
data_pred_mat = {}
for emo in data_tab["Emotion"]:
    emotion_mat = []
    emo_tab = data_tab[data_tab["Emotion"] == emo]
    for i in emo_tab.index:
        data = []
        data.append(emo_tab.loc[i,"Posit0"])
        data.append(emo_tab.loc[i,"Posit1"])
        data.append(emo_tab.loc[i,"Posit2"])
        data.append(emo_tab.loc[i,"Posit3"])
        data.append(emo_tab.loc[i,"story"])
        emotion_mat.append(data)
    data_pred_mat[emo] = np.array(emotion_mat)

In [8]:
import scipy

In [9]:
scipy.io.savemat("QPred.mat", quantum_pred_mat)

In [10]:
scipy.io.savemat("RawData.mat", data_pred_mat)

In [11]:
joint_table = {"quantum": quantum_pred_mat, "data": data_pred_mat,}

# Plots

In [12]:
def average_ratio_compression(mat):
    d = scipy.io.loadmat(mat)["parm"][2][0]
    return np.exp(d)

In [13]:
analysis_result = {}
for model in ["quantum", "data"]:
    analysis_result[model] = {}
    for emotion in data_tab["Emotion"].unique():
        mat = emotion.lower() + "_parm_" + model + ".mat"
        slope = scipy.io.loadmat(mat)["parm"][0][0]
        compression = average_ratio_compression(mat)
        analysis_result[model][emotion] = {"slope":slope, "compression":compression}

In [14]:
analysis_result["quantum"]

{'Embarrass': {'slope': -0.03860371985909852,
  'compression': 0.8630722737405184},
 'Happy': {'slope': -0.18232218014654142, 'compression': 0.9592025549885766},
 'Risk': {'slope': -0.3435222274078285, 'compression': 1.0053103408253279},
 'Sad': {'slope': -0.10467376161802935, 'compression': 0.9226768739350387},
 'Disgust': {'slope': -0.46981459177084717, 'compression': 0.9300343127169456}}

In [15]:
analysis_result["data"]

{'Embarrass': {'slope': -0.10973043672859636,
  'compression': 0.7937144312883826},
 'Happy': {'slope': -0.17466432988964525, 'compression': 0.9455013168138298},
 'Risk': {'slope': -0.31483836793966674, 'compression': 0.9227583703486146},
 'Sad': {'slope': -0.11518176137513204, 'compression': 0.8763627691346138},
 'Disgust': {'slope': -0.4824369739933071, 'compression': 0.873862880644033}}

In [16]:
def mean_model(mat,num_story):
    y_list = []
    parm = scipy.io.loadmat(mat)["parm"]
    x = np.linspace(0,3,100)
    for i in range(0,num_story):
        yretell = prediction(x,parm[0][0],parm[4+i][0],parm[2][0],parm[1][0])
        y_list.append(yretell)
    return y_list

In [17]:
def prediction(x,slope,alpha,d,intercept):
    y_retell = []
    for s in x:
        y = slope*s + alpha*np.exp(s*d) + intercept - 1
        y_retell.append(y)
    return y_retell

In [18]:
def spine(x,slope,intercept):
    y_retell = []
    for s in x:
        y = slope*s + intercept - 1
        y_retell.append(y)
    return y_retell

In [19]:
def spine_model(mat,num_story):
    parm = scipy.io.loadmat(mat)["parm"]
    x = np.linspace(0,3,100)
    y = spine(x,parm[0][0],parm[1][0])
    return y

In [20]:
prediction_plots = {}
for emotion in data_tab["Emotion"].unique():
    prediction_plots[emotion] = {}
    num_story = int(joint_table["quantum"][emotion][-1,4] + 1)
    mat = emotion.lower() + "_parm_" + "quantum" + ".mat"
    mean_pred = mean_model(mat,num_story)
    spines = spine_model(mat,num_story)
    prediction_plots[emotion] = {"pred": mean_pred, "spine": spines}

In [21]:
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.rcParams["figure.figsize"] = (8,6)
for emo in data_tab["Emotion"].unique():
    plt.title(emo,fontsize=40)
    for i in range(0,np.size(prediction_plots[emo]["pred"],0)):
        plt.plot(np.linspace(0,3,100),prediction_plots[emo]["pred"][i])
    plt.plot(np.linspace(0,3,100),prediction_plots[emo]["spine"], linewidth = 8, color = "red")
    plt.scatter(0.5, 8, color = "white", label = "exp(d) = " + str(round(analysis_result["quantum"][emo]["compression"],3)))
    plt.scatter(0.5, 8, color = "white", label = "slope = " + str(round(analysis_result["quantum"][emo]["slope"],3)))
    plt.ylim(-0.5,7.5)
    plt.rc('xtick', labelsize=20) 
    plt.rc('ytick', labelsize=20) 
    plt.legend(frameon=False,fontsize=25)
    plt.savefig(emo + "_mean_fig.jpg", dpi = 500)
    plt.close()